In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import threading

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipyparallel import Client 

from lib.oracles import LinearLMEOracle, LinearLMEOracleRegularized
from lib.solvers import LinearLMESolver, LinearLMERegSolver
from lib.problems import LinearLMEProblem

## LME Playground

In [58]:
study_sizes_widget = widgets.Text(value='100, 50, 10', placeholder="100, 100, 100", description='Study sizes:')
test_widget = widgets.IntSlider(min=1, max=20, value=10, step=1, description="Test size")
random_seed_widget = widgets.IntSlider(min=1, max=1000, value=212, step=1, description="Random seed")
num_features_widget = widgets.IntSlider(min=2, max=10, value=6, step=1, description="Features")
num_random_features_widget = widgets.IntSlider(min=2, max=10, value=2, step=1, description="Random effects")
how_close_widget = widgets.FloatSlider(min=0, max=1, value=1, step=0.01, description="How close?")
obs_std_widget = widgets.FloatLogSlider(min=-6, max=2, value=5e-2, step=1, description="Random noise std")
l2_beta_reg_widget = widgets.FloatLogSlider(min=-3, max=5, value=1, step=1, description="L2 beta regularization")
#l2_gamma_reg_widget = widgets.FloatLogSlider(min=-3, max=5, value=-3, step=1, description="L2 gamma regularization")
features_widget = widgets.HBox([num_features_widget, num_random_features_widget])
rest_widget = widgets.HBox([obs_std_widget, random_seed_widget])

mode_widget = widgets.ToggleButtons(options=['naive', 'fast'], 
                                         value='fast',
                                         description='Mode:')
method_widget = widgets.ToggleButtons(options=['GradDescent', 'NewtonRaphson', 'EM', 'VariableProjectionNR'], 
                                         value='NewtonRaphson',
                                         description='Method:')
bootstrap_widget = widgets.ToggleButtons(options=['None', 'Parametric', 'Nonparametric', 'Analytic'], 
                                         value='None',
                                         description='Bootstrap:')
bootstrap_capacity_widget = widgets.IntSlider(min=10, max=1000, value=100, step=10, description="Bootstrap samples")
launch_log = []

def visualize_sample_problem(study_sizes="100, 100, 100",
                 test_size=5,
                 num_features=6, 
                 num_random_effects=6,
                 how_close=1,
                 lb=0.1,
                 obs_std=0.1, 
                 bootstrap='Nonparametric',
                 bootstrap_capacity=100,
                 mode='fast',
                 method='NewtonRaphson',
                 random_seed=42):
    figsize=(16, 16)
    if study_sizes == "":
        study_sizes = [study1, study2, study3]
    else:
        study_sizes = [int(s) for s in study_sizes.split(", ")]
    test_study_sizes = [test_size]*len(study_sizes)
    
    beta = np.ones(num_features)
    beta[-1] = 0
    gamma = np.ones(num_random_effects)
    #gamma[-1] = 0.1
        
    train, beta, gamma, random_effects, errs = LinearLMEProblem.generate(study_sizes=study_sizes,
                                                                         num_features=num_features,
                                                                         beta=beta,
                                                                         gamma=gamma,
                                                                         num_random_effects=num_random_effects,
                                                                         how_close_z_to_x=how_close,
                                                                         obs_std=obs_std,
                                                                         seed=random_seed)
    
    empirical_gamma = np.sum(random_effects ** 2, axis=0) / len(study_sizes)
    
    test = LinearLMEProblem.generate(study_sizes=test_study_sizes, beta=beta, gamma=gamma,
                                     how_close_z_to_x=how_close,
                                     true_random_effects=random_effects,
                                     seed=random_seed + 1, 
                                     obs_std=obs_std,
                                     return_true_parameters=False)
    true_parameters = {
        "beta": beta,
        "gamma": gamma,
        "random_effects": random_effects,
        "errs": errs,
        "train": train,
        "test": test,
        "seed": random_seed
    }
    
    color_map = ["red", "green", "blue", "yellow", "black", "cyan", "purple", "orange"]
    
    tol = 1e-4
    
    
    if method == "VariableProjectionNR":
        train_oracle = LinearLMEOracleRegularized(train, lb=lb)
        test_oracle = LinearLMEOracleRegularized(test, lb=lb)
        model = LinearLMERegSolver(tol=tol, max_iter=100)
    else:    
        train_oracle = LinearLMEOracle(train)
        test_oracle = LinearLMEOracle(test)
        model = LinearLMESolver(tol=tol, max_iter=1000)
    
    logger = model.fit(train_oracle, test_oracle, method=method)
    if not logger["converged"]:
        print("Did not converge")    
    
        
    fig = plt.figure(figsize=figsize)
    grid = plt.GridSpec(4, 4, wspace=0.3, hspace=0.3)
    prediction_plot = fig.add_subplot(grid[:2, :2])


    model_parameters_plot = fig.add_subplot(grid[0, 2])
    pred_beta = model.beta
    model_parameters_plot.scatter(true_parameters['beta'], pred_beta)
    model_parameters_plot.set_xlabel("True parameters")
    model_parameters_plot.set_ylabel("Inferred parameters")
    model_parameters_low_lim = min(min(pred_beta), min(true_parameters['beta'])) - 0.1
    model_parameters_high_lim = max(max(pred_beta), max(true_parameters['beta'])) + 0.1
    model_parameters_plot.set_xlim(model_parameters_low_lim, model_parameters_high_lim)
    model_parameters_plot.set_ylim(model_parameters_low_lim, model_parameters_high_lim)
    model_parameters_plot.plot(model_parameters_plot.get_xlim(), model_parameters_plot.get_ylim(), ls='--', c=".3")
    
    random_effects_plot = fig.add_subplot(grid[1, 2])
    random_effects_low_lim = 0
    random_effects_high_lim = 0
    for i, (u_pred, u_true) in enumerate(zip(model.us, true_parameters['random_effects'])):
        random_effects_plot.scatter(u_true, u_pred, label="Study %d" % (i + 1), c=color_map[i])
        random_effects_low_lim = min(min(u_true), min(u_pred), random_effects_low_lim)
        random_effects_high_lim = max(max(u_true), max(u_pred), random_effects_high_lim)
    random_effects_plot.legend()
    random_effects_low_lim -= 0.1
    random_effects_high_lim += 0.1
    random_effects_plot.set_xlim(random_effects_low_lim, random_effects_high_lim)
    random_effects_plot.set_ylim(random_effects_low_lim, random_effects_high_lim)
    random_effects_plot.plot(random_effects_plot.get_xlim(), random_effects_plot.get_ylim(), ls='--', c=".3")
    random_effects_plot.set_xlabel("True random effects")
    random_effects_plot.set_ylabel("Inferred random effects")

    if method == "VariableProjectionNR":
        gamma_to_show = model.tgamma
    else:
        gamma_to_show = model.gamma

    gamma_plot = fig.add_subplot(grid[0, 3])
    gamma_plot.scatter(empirical_gamma, gamma_to_show, label="Inferred")
    gamma_low_lim = min(min(empirical_gamma), min(gamma_to_show)) - 0.2
    gamma_high_lim = max(max(empirical_gamma), max(gamma_to_show)) + 0.2
    gamma_plot.set_xlim(gamma_low_lim, gamma_high_lim)
    gamma_plot.set_ylim(gamma_low_lim, gamma_high_lim)
    gamma_plot.plot(gamma_plot.get_xlim(), gamma_plot.get_ylim(), ls='--', c=".3")
    gamma_plot.set_xlabel("Empirical gamma")
    gamma_plot.set_ylabel("Inferred gamma")

    abs_min = np.inf
    if num_random_effects == 2:
        trajectory_plot = fig.add_subplot(grid[2:4, 0:2])
    
        xlims = [0, 2] #ax.get_xlim()
        ylims = [0, 2] #ax.get_ylim()

        plot_resolution = 100
        x = np.linspace(0, xlims[1], plot_resolution)
        y = np.linspace(0, ylims[1], plot_resolution)
        z = np.zeros((plot_resolution, plot_resolution))
        zh = np.zeros((plot_resolution, plot_resolution))
        def psd(hessian):
            eigvals = np.linalg.eigvals(hessian)
            if np.linalg.norm(np.imag(eigvals)) > 1e-15:
                return -1
            min_eigval = min(np.real(eigvals))
            if min_eigval < 0:
                return -1
            else:
                return min_eigval
        for i, g2 in enumerate(y):
            for j, g1 in enumerate(x):
                gamma0 = np.array([g1, g2])
                beta0 = train_oracle.optimal_beta(gamma0)
                z[i, j] = train_oracle.loss(beta0, gamma0)
                hessian = train_oracle.hessian_gamma(beta0, gamma0)
                zh[i, j] = psd(hessian)

        abs_min = min(min(logger["loss"]), np.min(z)) - 1e-8

        csh = trajectory_plot.contourf(x, y, zh, levels=[0, 1e13], colors="lightgreen")

        levels = np.min(z) + np.array([1e-2, 1e-1, 1e0, 1e1, 1e2])
        levels_labels = ["1e-2", "1e-1", "1e0", "1e1", "1e2"]
        levels_dict = {levels[i]: levels_labels[i] for i in range(len(levels_labels))}

        cs = trajectory_plot.contour(x, y, z, levels=levels)
        plt.clabel(cs, fontsize=8, fmt=levels_dict)


        gamma_trace = np.array(logger["gamma"]).T
        trajectory_plot.plot(gamma_trace[0], gamma_trace[1], '-ob', label = method)

        trajectory_plot.set_xlim(xlims)
        trajectory_plot.set_ylim(ylims)
        trajectory_plot.set_xlabel(r"$\gamma_1$, variation of the first random effect")
        trajectory_plot.set_ylabel(r"$\gamma_2$, variation of the second random effect")
        
        extra_plot = fig.add_subplot(grid[2:4, 2:4])
        
        def psd_criterion(gamma_t):
            beta_t = train_oracle.optimal_beta(gamma_t)
            criterion_satisfied = []
            gamma_mat = np.diag(gamma_t)
            for x, y, z, l in train:
                omega = z.dot(gamma_mat).dot(z.T) + l
                xi = y - x.dot(beta_t)
                criterion_satisfied.append(xi.T.dot(np.linalg.inv(omega)).dot(xi) <= 1/2)
            return sum(criterion_satisfied)
        
        zc = np.zeros((plot_resolution, plot_resolution))
        for i, g2 in enumerate(y):
            for j, g1 in enumerate(x):
                gamma_t = np.array([g1, g2])
                zc[i, j] = psd_criterion(gamma_t)
                
        print(zc)
                
        cc = extra_plot.contourf(x, y, zc)
        cs = extra_plot.contour(x, y, z, levels=levels)
        plt.clabel(cs, fontsize=8, fmt=levels_dict)
        extra_plot.set_xlim(xlims)
        extra_plot.set_ylim(ylims)
        extra_plot.set_xlabel(r"$\gamma_1$, variation of the first random effect")
        extra_plot.set_ylabel(r"$\gamma_2$, variation of the second random effect")

    #loss_plot = fig.add_subplot(grid[1, 3])
    #min_loss = min(np.min(logger["loss"]), abs_min)
    #loss_plot.semilogy(range(len(logger["loss"])), logger["loss"] - min_loss + 1e-8, '-ob', label=method)
    #loss_plot.set_ylim((1e-8, 1e3))
    
    grad_plot = fig.add_subplot(grid[1, 3])
    grad_norm = [np.linalg.norm(gamma) for gamma in logger["grad_gamma"]]
    grad_plot.semilogy(range(len(logger["grad_gamma"])), grad_norm, '-ob', label=method)
    grad_plot.set_xlabel("Number of iteration")
    grad_plot.set_ylabel("Gradient norm")
    ylims = grad_plot.get_ylim()
    grad_plot.set_ylim(min(tol/10, ylims[0]), ylims[1])
    
    plt.show()
    
    print("True beta: %s"%(" ".join("%.2f"%s for s in beta.tolist())))
    print("Model beta: %s"%(" ".join("%.2f"%s for s in model.beta.tolist())))
    if method == "VariableProjectionNR":
        print("Model tbeta: %s"%(" ".join("%.2f"%s for s in model.tbeta.tolist())))
    print(" ")
    print("Empirical gamma: %s"%(" ".join("%.2f"%s for s in empirical_gamma.tolist())))
    print("Model gamma: %s"%(" ".join("%.2f"%s for s in model.gamma.tolist())))
    if method == "VariableProjectionNR":
        print("Model tgamma: %s"%(" ".join("%.2f"%s for s in model.tgamma.tolist())))
    
    launch_log.append(true_parameters)

In [59]:
interact_manual(visualize_sample_problem, 
                study_sizes = study_sizes_widget,
                test_size = test_widget,
                num_features = num_features_widget,
                num_random_effects = num_random_features_widget,
                how_close = how_close_widget,
                lb=l2_beta_reg_widget,
                obs_std = obs_std_widget,
                bootstrap = bootstrap_widget,
                bootstrap_capacity = bootstrap_capacity_widget,
                mode = mode_widget, 
                method = method_widget,
                random_seed = random_seed_widget)

interactive(children=(Text(value='100, 50, 10', description='Study sizes:', placeholder='100, 100, 100'), IntS…

<function __main__.visualize_sample_problem(study_sizes='100, 100, 100', test_size=5, num_features=6, num_random_effects=6, how_close=1, lb=0.1, obs_std=0.1, bootstrap='Nonparametric', bootstrap_capacity=100, mode='fast', method='NewtonRaphson', random_seed=42)>

## Feature Selection Playground

In [76]:
study_sizes_widget = widgets.Text(value='100, 50, 10', placeholder="100, 100, 100", description='Study sizes:')
test_widget = widgets.IntSlider(min=1, max=20, value=10, step=1, description="Test size")
random_seed_widget = widgets.IntSlider(min=1, max=1000, value=212, step=1, description="Random seed")
num_features_widget = widgets.IntSlider(min=2, max=10, value=6, step=1, description="Features")
num_random_features_widget = widgets.IntSlider(min=2, max=10, value=6, step=1, description="Random effects")
how_close_widget = widgets.FloatSlider(min=0, max=1, value=1, step=0.01, description="How close?")
obs_std_widget = widgets.FloatLogSlider(min=-6, max=2, value=5e-2, step=1, description="Random noise std")
l2_beta_reg_widget = widgets.FloatLogSlider(min=-3, max=5, value=1, step=1, description="L2 beta regularization")
#l2_gamma_reg_widget = widgets.FloatLogSlider(min=-3, max=5, value=-3, step=1, description="L2 gamma regularization")
features_widget = widgets.HBox([num_features_widget, num_random_features_widget])
rest_widget = widgets.HBox([obs_std_widget, random_seed_widget])

mode_widget = widgets.ToggleButtons(options=['naive', 'fast'], 
                                         value='fast',
                                         description='Mode:')
method_widget = widgets.ToggleButtons(options=['GradDescent', 'NewtonRaphson', 'EM', 'VariableProjectionNR'], 
                                         value='VariableProjectionNR',
                                         description='Method:')
bootstrap_widget = widgets.ToggleButtons(options=['None', 'Parametric', 'Nonparametric', 'Analytic'], 
                                         value='None',
                                         description='Bootstrap:')
bootstrap_capacity_widget = widgets.IntSlider(min=10, max=1000, value=100, step=10, description="Bootstrap samples")
launch_log = []

def visualize_sample_problem(study_sizes="100, 100, 100",
                 test_size=5,
                 num_features=6, 
                 num_random_effects=6,
                 how_close=1,
                 lb=0.1,
                 obs_std=0.1, 
                 bootstrap='Nonparametric',
                 bootstrap_capacity=100,
                 mode='fast',
                 method='NewtonRaphson',
                 random_seed=42):
    figsize=(16, 16)
    if study_sizes == "":
        study_sizes = [study1, study2, study3]
    else:
        study_sizes = [int(s) for s in study_sizes.split(", ")]
    test_study_sizes = [test_size]*len(study_sizes)
    
    beta = np.ones(num_features)
    beta[-1] = 0
    gamma = np.ones(num_random_effects)
    #gamma[-1] = 0.1
        
    train, beta, gamma, random_effects, errs = LinearLMEProblem.generate(study_sizes=study_sizes,
                                                                         num_features=num_features,
                                                                         beta=beta,
                                                                         gamma=gamma,
                                                                         num_random_effects=num_random_effects,
                                                                         how_close_z_to_x=how_close,
                                                                         obs_std=obs_std,
                                                                         seed=random_seed)
    
    empirical_gamma = np.sum(random_effects ** 2, axis=0) / len(study_sizes)
    
    test = LinearLMEProblem.generate(study_sizes=test_study_sizes, beta=beta, gamma=gamma,
                                     how_close_z_to_x=how_close,
                                     true_random_effects=random_effects,
                                     seed=random_seed + 1, 
                                     obs_std=obs_std,
                                     return_true_parameters=False)
    true_parameters = {
        "beta": beta,
        "gamma": gamma,
        "random_effects": random_effects,
        "errs": errs,
        "train": train,
        "test": test,
        "seed": random_seed
    }
    
    color_map = ["red", "green", "blue", "yellow", "black", "cyan", "purple", "orange"]
    
    tol = 1e-4
    
    
    if method == "VariableProjectionNR":
        train_oracle = LinearLMEOracleRegularized(train, lb=lb)
        test_oracle = LinearLMEOracleRegularized(test, lb=lb)
        model = LinearLMERegSolver(tol=tol, max_iter=100)
    else:    
        train_oracle = LinearLMEOracle(train)
        test_oracle = LinearLMEOracle(test)
        model = LinearLMESolver(tol=tol, max_iter=1000)
    
#     gamma_reg_upperbound = train_oracle.good_lambda_gamma(mode="upperbound")
#     gamma_reg_exact = train_oracle.good_lambda_gamma(mode="exact")
    gamma_reg_exact_full, g_opt = train_oracle.good_lambda_gamma(mode="exact_full_hess")
#     print("Upperbound: ", gamma_reg_upperbound)
#     print("Exact: ", gamma_reg_exact)
    print(r"$\lambda_\gamma$ adjustment: ", gamma_reg_exact_full)
#     print(g_opt)
#     print(gamma_reg_upperbound/gamma_reg_exact)
    

    train_oracle.lg = gamma_reg_exact_full
    test_oracle.lg = gamma_reg_exact_full

    parameters = {
        (num_features + 1, num_random_effects): (np.ones(num_features), np.zeros(num_random_effects) + 0.1, None)
    }

    for k in range(num_features, 0, -1):
        train_oracle.k = k
        test_oracle.k = k
        if k == num_features:
            train_oracle.lb = 0
            test_oracle.lb = 0
            tbeta = np.zeros(num_features)
        else:
            train_oracle.lb = lb
            test_oracle.lb = lb
            prev_beta, *rest = parameters[(k + 1, k)]
            tbeta = train_oracle.take_only_k_max(prev_beta, k)
        for j in range(k, 0, -1):
            train_oracle.j = j
            test_oracle.j = j
            if j == k:
                prev_beta, prev_gamma, *rest = parameters[(k + 1, j)]
            else:
                prev_beta, prev_gamma, *rest = parameters[(k, j + 1)]
            tgamma = train_oracle.take_only_k_max(prev_gamma, j)

            logger = model.fit(train_oracle,
                               test_oracle,
                               beta0=prev_beta,
                               gamma0=prev_gamma,
                               tbeta=tbeta,
                               tgamma=tgamma
                               )
            train_loss = train_oracle.loss_reg(model.beta, model.gamma, tbeta, tgamma)
            test_loss = test_oracle.loss_reg(model.beta, model.gamma, tbeta, tgamma)
            if not logger["converged"]:
                parameters[(k, j)] = (prev_beta, prev_gamma, tbeta, tgamma, logger, train_loss, test_loss)
            else:
                parameters[(k, j)] = (model.beta, model.gamma, model.tbeta, model.tgamma, logger, train_loss, test_loss) 
    
    
    train_error = np.zeros((num_features, num_features))
    test_error = np.zeros((num_features, num_features))
    for k in range(num_features, 0, -1):
        for j in range(k, 0, -1):
            beta, gamma, tbeta, tgamma, logger, train_loss, test_loss = parameters[(k, j)]
            train_error[k-1, j-1] = train_loss
            test_error[k-1, j-1] = test_loss
    
    fig = plt.figure(figsize=figsize)
    grid = plt.GridSpec(4, 4, wspace=0.3, hspace=0.3)
    train_error_plot = fig.add_subplot(grid[0:2, 0:2])
    train_error_plot.imshow(train_error)
    train_error_plot.set_xticks(np.arange(num_features))
    train_error_plot.set_yticks(np.arange(num_features))
    
    for k in range(num_features, 0, -1):
        for j in range(k, 0, -1):
            text = train_error_plot.text(j, k, "%.2f"%train_error[k-1, j-1], ha="center", va="center", color="w")
            
    print(train_error)
    print(test_error)

In [77]:
interact_manual(visualize_sample_problem, 
                study_sizes = study_sizes_widget,
                test_size = test_widget,
                num_features = num_features_widget,
                num_random_effects = num_random_features_widget,
                how_close = how_close_widget,
                lb=l2_beta_reg_widget,
                obs_std = obs_std_widget,
                bootstrap = bootstrap_widget,
                bootstrap_capacity = bootstrap_capacity_widget,
                mode = mode_widget, 
                method = method_widget,
                random_seed = random_seed_widget)

interactive(children=(Text(value='100, 50, 10', description='Study sizes:', placeholder='100, 100, 100'), IntS…

<function __main__.visualize_sample_problem(study_sizes='100, 100, 100', test_size=5, num_features=6, num_random_effects=6, how_close=1, lb=0.1, obs_std=0.1, bootstrap='Nonparametric', bootstrap_capacity=100, mode='fast', method='NewtonRaphson', random_seed=42)>